In [1]:
import numpy as np
import pandas as pd

# Loading the datasets
health = pd.read_csv("../Datasets/rankmd.csv", delimiter=";")
FastFood = pd.read_csv("../Datasets/FastFoodRestaurants.csv")
income = pd.read_csv("../Datasets/kaggle_income.csv", encoding="ISO 8859-1")

# Storing abbrevations of statenames
us_state_to_abbrev = {
    "Alabama": "AL",
    "Alaska": "AK",
    "Arizona": "AZ",
    "Arkansas": "AR",
    "California": "CA",
    "Colorado": "CO",
    "Connecticut": "CT",
    "Delaware": "DE",
    "Florida": "FL",
    "Georgia": "GA",
    "Hawaii": "HI",
    "Idaho": "ID",
    "Illinois": "IL",
    "Indiana": "IN",
    "Iowa": "IA",
    "Kansas": "KS",
    "Kentucky": "KY",
    "Louisiana": "LA",
    "Maine": "ME",
    "Maryland": "MD",
    "Massachusetts": "MA",
    "Michigan": "MI",
    "Minnesota": "MN",
    "Mississippi": "MS",
    "Missouri": "MO",
    "Montana": "MT",
    "Nebraska": "NE",
    "Nevada": "NV",
    "New Hampshire": "NH",
    "New Jersey": "NJ",
    "New Mexico": "NM",
    "New York": "NY",
    "North Carolina": "NC",
    "North Dakota": "ND",
    "Ohio": "OH",
    "Oklahoma": "OK",
    "Oregon": "OR",
    "Pennsylvania": "PA",
    "Rhode Island": "RI",
    "South Carolina": "SC",
    "South Dakota": "SD",
    "Tennessee": "TN",
    "Texas": "TX",
    "Utah": "UT",
    "Vermont": "VT",
    "Virginia": "VA",
    "Washington": "WA",
    "West Virginia": "WV",
    "Wisconsin": "WI",
    "Wyoming": "WY",
    "District of Columbia": "DC",
    "American Samoa": "AS",
    "Guam": "GU",
    "Northern Mariana Islands": "MP",
    "Puerto Rico": "PR",
    "United States Minor Outlying Islands": "UM",
    "U.S. Virgin Islands": "VI",
}
    
# Inverting the dictionary
abbrev_to_us_state = dict(map(reversed, us_state_to_abbrev.items()))

# Constructing a state dataset
FastFood['State'] = FastFood['province'].map(abbrev_to_us_state)
States = health.copy()
# Zero padding FIPS to fit length og 5
States = States[States['FIPS'].astype(str).str.endswith('000')] 
health['FIPS']=health['FIPS'].apply(lambda x: '{0:0>5}'.format(x))

# Changing FEI to be of type float 
health["food_environment_index_Food Environment Index"] = health["food_environment_index_Food Environment Index"].str.replace(",",".").astype(float)

In [2]:
# Removing 'County' from the county names in income dataset
income["County"] = income.County.str.replace(' County', '')

# Merging income and health data 
temp_df = income.groupby(["State_Name","County"]).mean().reset_index()
new_df = pd.merge(health.copy(), temp_df.copy(),  how='left', left_on=['State','County'], right_on = ['State_Name','County'])

In [3]:
#ONLY NEEDS TO BE RAN ONCE AS THE COUNTIES ARE STORED IN THE CSV.

#!{sys.executable} -m pip install pgeocode
#import pgeocode

#nomi = pgeocode.Nominatim('us')
#county_names = []
#for i in range(len(FastFood)):
#    county_names.append(nomi.query_postal_code(FastFood["postalCode"][i]).county_name)
    
#FastFood["County"] = county_names
#FastFood.to_csv("../Datasets/FastFoodRestaurants.csv")

In [4]:
# Merging Fastfood with health and income data
temptemp = FastFood.groupby(["State", "County"]).count().reset_index()[['State','County','address']]
tempo = temptemp.rename(columns={'address':'nr of FFchains'})
data_df = pd.merge(new_df, tempo,  how='left', left_on=['State','County'], right_on =['State','County'])
data_df['nr of FFchains'] = data_df['nr of FFchains'].fillna(0)

data = data_df[["FIPS", "State", "County", "premature_deathYears_of_Potential_Life_Lost_Rate",'adult_obesity_% Adults with Obesity',
                 "adult_smoking_% Smokers", "excessive_drinking_% Excessive Drinking", "food_environment_index_Food Environment Index",
                 "uninsured_% Uninsured", "unemployed_% Unemployed", 'nr of FFchains', 'Mean']]
data = data.dropna()
data['is_obese'] = data['adult_obesity_% Adults with Obesity']>=33
data = data.drop(['adult_obesity_% Adults with Obesity'],axis=1)
data["unemployed_% Unemployed"] = data["unemployed_% Unemployed"].str.replace(",",".").astype(float).astype(int)

In [5]:
data.to_csv("../Datasets/Mixed_data.csv")

In [6]:
data

,FIPS,State,County,premature_deathYears_of_Potential_Life_Lost_Rate,adult_smoking_% Smokers,excessive_drinking_% Excessive Drinking,food_environment_index_Food Environment Index,uninsured_% Uninsured,unemployed_% Unemployed,nr of FFchains,Mean,is_obese
1,01001,Alabama,Autauga,7830.0,20,14,6.7,10.0,2,0.0,53735.557235,True
3,01005,Alabama,Barbour,11477.0,26,12,5.5,14.0,3,1.0,37725.000000,True
5,01009,Alabama,Blount,11096.0,23,16,7.9,14.0,2,3.0,55127.000000,True
7,01013,Alabama,Butler,12848.0,23,13,6.8,12.0,3,0.0,27993.000000,True
9,01017,Alabama,Chambers,10371.0,23,14,6.7,12.0,2,4.0,45107.000000,True
...,...,...,...,...,...,...,...,...,...,...,...,...
3185,56031,Wyoming,Platte,8616.0,20,21,7.6,12.0,3,2.0,127999.000000,False
3186,56033,Wyoming,Sheridan,6136.0,18,21,7.6,11.0,3,3.0,68733.000000,False
3188,56037,Wyoming,Sweetwater,7775.0,19,19,7.5,11.0,3,2.0,0.000000,False
3190,56041,Wyoming,Uinta,8081.0,21,18,7.3,12.0,3,2.0,89130.000000,True
